# Thesis Notes

## Feature Engineering

The data is actually very large (multiple GB of image data in total for only the satellite images from the [NOAA](https://www.ngdc.noaa.gov/eog/dmsp/downloadV4composites.html#AVSLCFC).

__But__: The number of observations (_n_) given naturally is actually not that big since only yearly data from 1992 onwards is available. But we do have a large number of features (_m_) given by the size of the images (~ 16kx43k). To use the data in a meaningful way it makes sense to engineer the features to give more information about the connection we want to study (natural disasters => luminosity).